In [3]:
import numpy as np
from astropy import units as u
from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import build_default_calc

from tqdm import tqdm
from matplotlib import pyplot as plt


## Survey Depth and Speed Estimation with `build_survey`

This notebook estimates the **5-sigma detection depth** and **survey speed** for an ultrawide Roman survey.

### Inputs

1. **Exposure time (per dither)**  
   Will be matched to the closest C2A_IMG_HLWAS MA table value.

2. **Number of dithers**  
   The number of exposures per pointing.

3. **Filter**  
   The imaging filter to use (e.g., `f158`, `f146`).

4. **Field**  
   Options are:
   - `"high"` for Galactic latitude |b| > 30°  
   - Any other value is assumed to target |b| < 30°

### Outputs
5-sigma depth (AB mag), Survey Speed (sq deg/day), Survey Overhead rate (%), and Number of Days to cover the specified field.


In [2]:
def build_survey(texp=60,
                 ndither=3,
                 filt='f158',
                 field='high'):
    
    calc = build_default_calc('roman', 'wfi', 'imaging')
    calc['configuration']['instrument']['filter'] = filt
    
    # Assume config for C2A_IMG_HLWAS
    # Exposure times are not infinitely adjustable; need to be within the MA table.
    c2a_exposures = np.asarray([28.46, 53.762, 79.06,
                                104.36, 135.99, 139.15])
    c2a_nresultants = np.arange(4,10,1)
    nresult = c2a_nresultants[np.argmin(np.abs(c2a_exposures-texp))]
    
    # Quirk: NRESULTANTS IS +1 FROM MA TABLES!!!
    calc['configuration']['detector'] = {
        # 'readmode': 'c2a_img_hlwas',
        'ma_table_name': 'C2a_IMG_HLWAS',
        'subarray': 'full',
        'nexp': ndither,
        'nint': 1,
        'nresultants': nresult+1
    }
    calc['background'] = 'minzodi'
    calc['background_level'] = 'medium'
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
    
    # Now seach over grid to find approximate 5-sigma depth
    mag_range = np.arange(23, 27, 0.1)
    sigma_array = np.zeros_like(mag_range,dtype=np.float32)
    for i, mag in tqdm(enumerate(mag_range), total=len(mag_range)):
        # # Optional: Modify SED shape for a Gaia source
        # calc['scene'][0]['spectrum']['sed']['sed_type'] = 'phoenix'
        # calc['scene'][0]['spectrum']['sed']['key'] = 'k0v'
        # calc['scene'][0]['spectrum']['normalization'] = {
        #     'type': 'photsys',           
        #     'norm_flux': mag,            # e.g., G = 21
        #     'norm_fluxunit': 'abmag',
        #     'bandpass': 'gaia,g',
        # }
        calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
        report = perform_calculation(calc)
        sigma_array[i] = report['scalar']['sn']

    sig5_index = np.argmin(np.abs(5-sigma_array))
    
    # Now calculate survey speed
    science_time = report['scalar']['total_exposure_time']
    slew_time = (18.9*(ndither-1) + 39.3)
    survey_rate = 0.28*(24*u.hour.to(u.second))/(science_time + slew_time)
    survey_overhead = slew_time/(slew_time+science_time)
    allsky_sqdeg = (4*np.pi*u.sr).to(u.deg**2).value
    print(f"Selected Survey Parameters: {filt:s}, {science_time/ndither:.2f}s x {ndither:d} = {science_time:.2f}s total exposure time.")
    print("Key Outputs:")
    print(f"\t5-sigma depth: {mag_range[sig5_index]:.1f} mag")
    print(f"\tSurvey Speed: {survey_rate:.2f} sq deg/day")
    print(f"\tSurvey Overhead: {survey_overhead*100:.2f} %")
    if field=='high':
        print(f"\tDays to cover |b|>30: {(allsky_sqdeg/2-5000)/survey_rate:.1f}")
    else:
        print(f"\tDays to cover |b|<30: {(allsky_sqdeg/2)/survey_rate:.1f} days")
        
build_survey(texp=70,ndither=2,filt='f158',field='high')

100%|█████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.68it/s]

Selected Survey Parameters: f158, 82.22s x 2 = 164.45s total exposure time.
Key Outputs:
	5-sigma depth: 25.1 mag
	Survey Speed: 108.66 sq deg/day
	Survey Overhead: 26.14 %
	Days to cover |b|>30: 143.8


In [292]:
build_survey(texp=100,ndither=2,filt='f158',field='high')

100%|█████████████████████████████████████████████████| 40/40 [00:07<00:00,  5.30it/s]

Selected Survey Parameters: f158, 107.52s x 2 = 215.05s total exposure time.
Key Outputs:
	5-sigma depth: 25.4 mag
	Survey Speed: 88.53 sq deg/day
	Survey Overhead: 21.30 %
	Days to cover |b|>30: 176.5
